In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np
import csv
import requests
import os


/tmp/ipykernel_10438/712037525.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Supondo que 'dados' seja o seu DataFrame com os dados categóricos
# e 'colunas_categoricas' seja uma lista das colunas que são categóricas
dados = pd.read_csv("styles.csv")

colunas_categoricas = ["gender","masterCategory","subCategory","articleType","baseColour","season","year","usage"]
# Selecionar apenas as colunas categóricas
dados_categoricos = dados[colunas_categoricas]

# Selecionar a primeira coluna (ID)
id_coluna = dados[['id']]

# Criar um codificador one-hot
encoder = OneHotEncoder()

# Ajustar e transformar os dados categóricos
dados_codificados = encoder.fit_transform(dados_categoricos)

# Converter a saída para um DataFrame pandas
dados_codificados_df = pd.DataFrame(dados_codificados.toarray(), columns=encoder.get_feature_names_out(colunas_categoricas))


# Adicionar a coluna de ID de volta aos dados
dados_codificados_df.insert(0, 'id', id_coluna)

# Agora você tem um DataFrame com as colunas categóricas codificadas one-hot,
# mantendo a primeira coluna (ID) intacta


In [3]:
# Supondo que você tenha o número de clusters desejado armazenado na variável 'num_clusters'
num_clusters = 15

# Inicializar o algoritmo K-means com o número de clusters desejado
kmeans = KMeans(n_clusters=num_clusters)

# Ajustar o modelo aos dados codificados
modelo = kmeans.fit(dados_codificados)

# Obter os rótulos dos clusters para cada observação
rotulos_clusters = modelo.labels_
centroides = modelo.cluster_centers_
# Você pode agora usar os rótulos dos clusters para análise posterior ou visualização


In [4]:
print(centroides)
print(len(rotulos_clusters))

[[1.07781850e-01 7.24294029e-02 1.69864195e-01 ... 9.70036646e-03
  0.00000000e+00 3.66458288e-03]
 [1.22422680e-02 1.01481959e-02 8.05251289e-01 ... 1.14207474e-01
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 3.99361022e-04 3.99361022e-02 ... 9.98402556e-03
  4.79233227e-03 1.59744409e-03]
 ...
 [1.74672489e-02 7.64192140e-03 5.41484716e-01 ... 7.64192140e-03
  0.00000000e+00 0.00000000e+00]
 [5.06450072e-02 6.06784520e-02 4.16626851e-01 ... 1.79168657e-01
  0.00000000e+00 9.55566173e-04]
 [1.72413793e-03 2.10727969e-03 2.72413793e-01 ... 5.63218391e-02
  2.49042146e-03 3.44827586e-03]]
44446


In [5]:
novos_dados = ["Women",'Personal Care','Sandal',"Sandals","Pink",'Summer',2017,"Casual"]

novos_dados_df = pd.DataFrame([novos_dados])

# Aplicar o codificador aos novos dados
novos_dados_codificados = encoder.transform(novos_dados_df).toarray()




# Encontre os vizinhos mais próximos do novo dado
num_vizinhos = 5  # Número de vizinhos mais próximos que você deseja encontrar
vizinhos_mais_proximos = NearestNeighbors(n_neighbors=num_vizinhos, algorithm='ball_tree').fit(centroides)
distancias_clusters, indices_clusters = vizinhos_mais_proximos.kneighbors(novos_dados_codificados)


pontos_cluster = []
for indice_cluster in indices_clusters.flatten():
    pontos_cluster.extend(modelo.labels_ == indice_cluster)

# Calcular as distâncias dos pontos individuais para o novo dado
pontos_individuais = dados_codificados[pontos_cluster]
vizinhos_mais_proximos = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(dados_codificados)
distancias, indices = vizinhos_mais_proximos.kneighbors(novos_dados_codificados)

print(indices, distancias)
for indice in indices:
    ids = dados.loc[indice, 'id'].tolist()
print(ids)

[[319  79 416 188 594 320 558 636 379  66]] [[2.44948974 2.44948974 2.44948974 2.44948974 2.44948974 2.44948974
  2.44948974 2.44948974 2.44948974 2.44948974]]
[44173, 8574, 43994, 58980, 55496, 25978, 58973, 57597, 55805, 43993]


/home/codespace/.local/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/neighbors/_base.py:584: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


In [8]:
def baixar_imagem(url, nome_arquivo, i):
    try:
        # Faz uma requisição GET para a URL da imagem
        resposta = requests.get(url)

        # Verifica se a requisição foi bem-sucedida
        if resposta.status_code == 200:
            # Abre o arquivo no modo de escrita binária ('wb') e escreve o conteúdo da resposta
            with open(f'{nome_arquivo}{i}.jpg', 'wb') as arquivo:
                arquivo.write(resposta.content)
            print(f"Imagem salva como {nome_arquivo}")
        else:
            print("Falha ao baixar a imagem:", resposta.status_code)
    except Exception as e:
        print("Erro ao baixar a imagem:", str(e))

In [10]:
os.mkdir( "/workspaces/cabid/Roupas_semelhantes/")
with open("images.csv", 'r', newline='', encoding='utf-8') as csv_file:
  leitor_csv = csv.reader(csv_file)
  cont = 0
  for linha in leitor_csv:
    linha_id = linha[0][:-4]
    for id in ids:
      if id == int(linha_id):
        baixar_imagem(linha[1], "/workspaces/cabid/Roupas_semelhantes/", cont)
        cont +=1
  

    

Imagem salva como /workspaces/cabid/Roupas_semelhantes/
Imagem salva como /workspaces/cabid/Roupas_semelhantes/
Imagem salva como /workspaces/cabid/Roupas_semelhantes/
Imagem salva como /workspaces/cabid/Roupas_semelhantes/
Imagem salva como /workspaces/cabid/Roupas_semelhantes/
Imagem salva como /workspaces/cabid/Roupas_semelhantes/
Imagem salva como /workspaces/cabid/Roupas_semelhantes/
Imagem salva como /workspaces/cabid/Roupas_semelhantes/
Imagem salva como /workspaces/cabid/Roupas_semelhantes/
Imagem salva como /workspaces/cabid/Roupas_semelhantes/


### Esta celula foi utilizada para apagar a última coluna.
Foi preciso apagar por que cada valor era uma frase e em alguns casos(quando havia uma virgula na frase) esbarrava em um erro ao tentar abrir o csv com pandas
Em segundo lugar aquele tipo de dados seria dificilmente bem processado pelo cluster
este trecho de código merece ser esquecido e apagado de nossas memórias, está aqui apenas para o caso de precisarmos dele de novo.

In [ ]:
import csv

# Lista para armazenar as linhas do arquivo CSV sem a última coluna
linhas_sem_ultima_coluna = []

with open('new_styles.csv', 'r', newline='') as csvfile:
    leitor_csv = csv.reader(csvfile)
    for linha in leitor_csv:
        try:
            print(linha[9])
            linha_sem_ultima_coluna = linha[:-1] 
            # Adicionar a linha sem a última coluna à lista
            linhas_sem_ultima_coluna.append(linha_sem_ultima_coluna)
        except:
            linhas_sem_ultima_coluna.append(linha)
            continue
            
       

# Escrever as linhas sem a última coluna em um novo arquivo CSV
with open('new_styles2.csv', 'w', newline='') as csvfile:
    escritor_csv = csv.writer(csvfile)
    escritor_csv.writerows(linhas_sem_ultima_coluna)

In [5]:
print(dados_categoricos)

      gender masterCategory subCategory            articleType baseColour   
0        Men        Apparel     Topwear                 Shirts  Navy Blue  \
1        Men        Apparel  Bottomwear                  Jeans       Blue   
2      Women    Accessories     Watches                Watches     Silver   
3        Men        Apparel  Bottomwear            Track Pants      Black   
4        Men        Apparel     Topwear                Tshirts       Grey   
...      ...            ...         ...                    ...        ...   
44441    Men       Footwear       Shoes           Casual Shoes      White   
44442    Men       Footwear  Flip Flops             Flip Flops        Red   
44443    Men        Apparel     Topwear                Tshirts       Blue   
44444  Women  Personal Care   Fragrance  Perfume and Body Mist       Blue   
44445  Women    Accessories     Watches                Watches       Pink   

       season    year   usage  
0        Fall  2011.0  Casual  
1      Summ